In [6]:
from os import listdir
import os
import cv2
import numpy as np
import xml.etree.ElementTree as ET
import imgaug as ia
from imgaug import augmenters as iaa
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from files import *
from pascal_voc_writer import Writer
#from files import *

In [7]:
#img=cv2.imread('./OD/image/image_001.jpg')
#print(img)
 
#plt.imshow(img)
#plt.show()

In [8]:
def read_anntation(xml_file: str):
    tree = ET.parse(xml_file)
    root = tree.getroot()

    bounding_box_list = []

    file_name = root.find('filename').text
    for obj in root.iter('object'):

        object_label = obj.find("name").text
        for box in obj.findall("bndbox"):
            x_min = int(box.find("xmin").text)
            y_min = int(box.find("ymin").text)
            x_max = int(box.find("xmax").text)
            y_max = int(box.find("ymax").text)

        bounding_box = [object_label, x_min, y_min, x_max, y_max]
        bounding_box_list.append(bounding_box)

    return bounding_box_list, file_name

In [12]:
def read_train_dataset(dir):
    images = []
    annotations = []
    ldir = 'C:/project/data3/'
    for file in listdir(dir):
      
        if 'jpg' in file.lower() or 'png' in file.lower():
            images.append(cv2.imread(dir + file, 1))
            annotation_file = file.replace(file.split('.')[-1], 'xml')
            bounding_box_list, file_name = read_anntation(ldir + annotation_file)
            annotations.append((bounding_box_list, annotation_file, file_name))

    images = np.array(images)

    return images, annotations

In [14]:
#imgaug의 글로벌 RNG(무작위 제어)의 seed 설정
ia.seed(1)

#r경로 지정
dir = 'C:/project/data3/'
dir1 = 'C:/project/data2/'
#경로에 있는 이미지와 xml 정보를 읽음
images, annotations = read_train_dataset(dir)

#이미지의 개수만큼 반복
for idx in range(len(images)):
    
    #해당 인덱스의 이미지
    image = images[idx]
    #해당 인덱스의 바운딩 박스
    boxes = annotations[idx][0]

    #바운딩 박스 정보를 저장할 리스트
    ia_bounding_boxes = []
    for box in boxes:
        #바운딩 박스들을 리스트에 저장
        ia_bounding_boxes.append(ia.BoundingBox(x1=box[1], y1=box[2], x2=box[3], y2=box[4]))
    #바운딩 박스들을 이미지 위에 그림
    bbs = ia.BoundingBoxesOnImage(ia_bounding_boxes, shape=image.shape)

    #크기 지정
    seq = iaa.Sequential([
        iaa.Resize(0.7)
        #iaa.Multiply((1.5,1.5))
        #iaa.Affine(
            #shear=(-15,15)
            #rotate=45
        #)
    ])

    #이미지를 지정한 크기로 변경
    seq_det = seq.to_deterministic()

    #이미지와 바운딩 박스의 크기 변경
    image_aug = seq_det.augment_images([image])[0]
    bbs_aug = seq_det.augment_bounding_boxes([bbs])[0]

    #크기를 변경하기 전 이미지와변경한 후 이미지를 비교
    for i in range(len(bbs.bounding_boxes)):
        before = bbs.bounding_boxes[i]
        after = bbs_aug.bounding_boxes[i]
        print("BB %d: (%.4f, %.4f, %.4f, %.4f) -> (%.4f, %.4f, %.4f, %.4f)" % (
            i,
            before.x1, before.y1, before.x2, before.y2,
            after.x1, after.y1, after.x2, after.y2)
        )
        
    image_before = bbs.draw_on_image(image, size=5)
    image_after = bbs_aug.draw_on_image(image_aug, size=5, color=[0, 0, 255])

    #cv2.imshow('image_before', cv2.resize(image_before, (380, 640)))
    #cv2.imshow('image_after', cv2.resize(image_after, (380, 640)))
    
    cv2.imwrite
    
    #변경하기 전 이미지와 변경한 후 이미지를 plot으로 표현하여 출력
    fig = plt.figure()
    rows = 1
    cols = 2
    
    
    
    aimage = fig.add_subplot(rows, cols, 1)
    plt.imshow(image_before)
    
    bimage = fig.add_subplot(rows, cols, 2)
    plt.imshow(image_after)
    plt.show()
    
    image_aug = seq_det.augment_images([image])[0]
    bbs_aug = seq_det.augment_bounding_boxes([bbs])[0]

    new_image_file = dir1 + '0.7resize_' + annotations[idx][2]
    cv2.imwrite(new_image_file, image_aug)

    h, w = np.shape(image_aug)[0:2]
    voc_writer = Writer(new_image_file, w, h)

    for i in range(len(bbs_aug.bounding_boxes)):
        bb_box = bbs_aug.bounding_boxes[i]
        voc_writer.addObject(boxes[i][0], int(bb_box.x1), int(bb_box.y1), int(bb_box.x2), int(bb_box.y2))

    voc_writer.save(dir1 + '0.7resize_' + annotations[idx][1])
    
    #cv2.waitKey(0)

NameError: name 'annotations' is not defined